In [1]:
import pandas as pd

In [2]:
! wget https://s3.amazonaws.com/imcbucket/data/flights/2008.csv

--2022-01-30 01:50:07--  https://s3.amazonaws.com/imcbucket/data/flights/2008.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.93.221
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.93.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [binary/octet-stream]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M  16.0MB/s    in 43s     

2022-01-30 01:50:51 (15.2 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [3]:
! wc -l 2008.csv

7009729 2008.csv


In [4]:
f = "2008.csv"
num_lines = sum(1 for l in open(f))
size = int(num_lines / 100)
import random
skip_idx = random.sample(range(1, num_lines), num_lines - size)
raw_pd = pd.read_csv(f, skiprows=skip_idx)

In [5]:
#raw_pd = pd.read_csv("2008.csv")

In [6]:
raw_pd.head(5)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,617.0,615,652.0,650,WN,11,N689SW,95.0,95.0,70.0,2.0,2.0,IND,MCI,451,6.0,19.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,2248.0,2125,102.0,2345,WN,890,N618WN,74.0,80.0,60.0,77.0,83.0,LAS,ABQ,487,4.0,10.0,0,NaN,0,7.0,0.0,0.0,0.0,70.0
2,2008,1,3,4,934.0,935,1453.0,1505,WN,830,N436WN,199.0,210.0,177.0,-12.0,-1.0,LAS,MDW,1521,6.0,16.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,2237.0,2140,2335.0,2245,WN,2454,N352SW,58.0,65.0,47.0,50.0,57.0,LAX,LAS,236,3.0,8.0,0,NaN,0,3.0,0.0,0.0,0.0,47.0
4,2008,1,3,4,1803.0,1730,1933.0,1845,WN,440,N720WN,90.0,75.0,67.0,48.0,33.0,LAX,OAK,337,5.0,18.0,0,NaN,0,6.0,0.0,15.0,0.0,27.0


In [7]:
raw_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70096 entries, 0 to 70095
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               70096 non-null  int64  
 1   Month              70096 non-null  int64  
 2   DayofMonth         70096 non-null  int64  
 3   DayOfWeek          70096 non-null  int64  
 4   DepTime            68695 non-null  float64
 5   CRSDepTime         70096 non-null  int64  
 6   ArrTime            68547 non-null  float64
 7   CRSArrTime         70096 non-null  int64  
 8   UniqueCarrier      70096 non-null  object 
 9   FlightNum          70096 non-null  int64  
 10  TailNum            69264 non-null  object 
 11  ActualElapsedTime  68519 non-null  float64
 12  CRSElapsedTime     70084 non-null  float64
 13  AirTime            68519 non-null  float64
 14  ArrDelay           68519 non-null  float64
 15  DepDelay           68695 non-null  float64
 16  Origin             700

In [8]:
def t_timeperiod(origin):
    if origin is None:
        period = None
    elif origin > 0 and origin < 600:
        period = '00.01-05.59'
    elif origin >= 600 and origin <=1200:
        period = '06.00-11.59'
    elif origin >= 1200 and origin <= 1800:
        period = '12.00-17.59'
    elif origin >= 1800 and origin <= 2400:
        period = '18.00-24.00'
    else:
        period = 'NA'
    return period

In [9]:
raw_pd['DepTime'] = raw_pd['DepTime'].apply(lambda x:t_timeperiod(x))

In [10]:
max_distance = raw_pd['Distance'].max()
min_distance = raw_pd['Distance'].min()

In [11]:
max_ArrDelay = raw_pd['ArrDelay'].max()
min_ArrDelay = raw_pd['ArrDelay'].min()

In [12]:
def t_normalized_distance(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_distance)/(max_distance-min_distance))

In [13]:
def t_normalized_ArrDelay(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_ArrDelay)/(max_ArrDelay-min_ArrDelay))

In [14]:
raw_pd['Distance'] = raw_pd['Distance'].apply(lambda x:t_normalized_distance(x))

In [15]:
raw_pd['ArrDelay'] = raw_pd['ArrDelay'].apply(lambda x:t_normalized_ArrDelay(x))

In [16]:
features_pd = raw_pd[['UniqueCarrier','Origin','Dest',\
        'DepTime','TaxiOut','TaxiIn','DepDelay',\
        'DayOfWeek','Distance','ArrDelay']]

In [17]:
final_pd = features_pd.dropna()

In [18]:
final_pd.count()

UniqueCarrier    68519
Origin           68519
Dest             68519
DepTime          68519
TaxiOut          68519
TaxiIn           68519
DepDelay         68519
DayOfWeek        68519
Distance         68519
ArrDelay         68519
dtype: int64

In [19]:
features_pd.head(20)

,UniqueCarrier,Origin,Dest,DepTime,TaxiOut,TaxiIn,DepDelay,DayOfWeek,Distance,ArrDelay
0,WN,IND,MCI,06.00-11.59,19.0,6.0,2.0,4,0.085175,0.084222
1,WN,LAS,ABQ,18.00-24.00,10.0,4.0,83.0,4,0.092476,0.164179
2,WN,LAS,MDW,06.00-11.59,16.0,6.0,-1.0,4,0.302170,0.069296
3,WN,LAX,LAS,18.00-24.00,8.0,3.0,57.0,4,0.041574,0.135394
4,WN,LAX,OAK,18.00-24.00,18.0,5.0,33.0,4,0.062056,0.133262
5,WN,MCO,BDL,12.00-17.59,9.0,4.0,3.0,4,0.206652,0.091684
6,WN,MDW,CLE,18.00-24.00,9.0,6.0,69.0,4,0.055972,0.153518
7,WN,MSY,BNA,06.00-11.59,8.0,6.0,0.0,4,0.089231,0.089552
8,WN,OKC,DEN,06.00-11.59,8.0,5.0,-2.0,4,0.094099,0.073561
9,WN,OMA,MDW,12.00-17.59,8.0,4.0,108.0,4,0.079497,0.191898


In [20]:


#raw_pd.isnull().sum()
#nullCol_series = raw_pd.isnull().sum()
#notNull_list = list(nullCol_series[nullCol_series == 0].index)
#notNull_list
#requiredCol_list = ['TaxiIn','TaxiOut','ArrDelay']
#selectedCol_list = notNull_list + requiredCol_list
#noNull_pd = raw_pd.loc[:,selectedCol_list].dropna()
#noNull_pd.isnull().sum()
#noNull_pd.describe()
#noNull_pd.info()

In [21]:
#final_pd = noNull_pd

In [22]:
import pandas as pd
from seaborn import load_dataset
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [23]:
train_df = final_pd.sample(frac=0.8,random_state=123)

In [24]:
test_df = final_pd.drop(train_df.index)

In [25]:
train_df.count()

UniqueCarrier    54815
Origin           54815
Dest             54815
DepTime          54815
TaxiOut          54815
TaxiIn           54815
DepDelay         54815
DayOfWeek        54815
Distance         54815
ArrDelay         54815
dtype: int64

In [26]:
test_df.count()

UniqueCarrier    13704
Origin           13704
Dest             13704
DepTime          13704
TaxiOut          13704
TaxiIn           13704
DepDelay         13704
DayOfWeek        13704
Distance         13704
ArrDelay         13704
dtype: int64

In [27]:
train_labels = train_df.filter(['ArrDelay'])
train_df = train_df.loc[:, train_df.columns != 'ArrDelay']

In [28]:
type(train_labels)

pandas.core.frame.DataFrame

In [29]:
test_labels = test_df.filter(['ArrDelay'])
test_df = test_df.loc[:, test_df.columns != 'ArrDelay']

In [30]:
X_train = train_df
X_test = test_df
y_train = train_labels
y_test = test_labels

In [31]:
categorical = list(X_train.select_dtypes('object').columns)
print(f"Categorical columns are: {categorical}")

numerical = list(X_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")

Categorical columns are: ['UniqueCarrier', 'Origin', 'Dest', 'DepTime']
Numerical columns are: ['TaxiOut', 'TaxiIn', 'DepDelay', 'DayOfWeek', 'Distance']


In [32]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz, DecisionTreeRegressor
from sklearn import tree
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [33]:
# Define custom transformer
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

# Define categorical pipeline
cat_pipe = Pipeline([('selector', ColumnSelector(categorical)),
                     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline([('selector', ColumnSelector(numerical))
                     #('imputer', SimpleImputer(strategy='median')),
                     #('scaler', MinMaxScaler())
                    ])

# Combine categorical and numerical pipeline
preprocessor = FeatureUnion(transformer_list=[('cat', cat_pipe),
                                              ('num', num_pipe)])

# Model Training
pipe = Pipeline(steps=[('preprocessor', preprocessor),
              ('model', DecisionTreeRegressor(max_depth=None))])
pipeline_model = pipe.fit(X_train, y_train)

In [34]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

predictions = pipeline_model.predict(X_test)
print("\t RMSE:", mean_squared_error(y_test, predictions, squared=False))
print("\t R2 score:", r2_score(y_test, predictions))

	 RMSE: 0.013844536347814426
	 R2 score: 0.8741306473632185
